In [6]:
import pandas as pd
import json, re, unicodedata
from typing import Tuple, Optional

# 정규식
LAW_NAME_RE = re.compile(r"([가-힣A-Za-z]+법)")
ARTICLE_RE  = re.compile(r"제(\d+)조")

def normalize(s: str) -> str:
    """유니코드 정규화 + 공백 제거(필요시 공백 유지해도 됨)"""
    return re.sub(r"\s+", "", unicodedata.normalize("NFKC", s or ""))

def extract_digits_to_int(s: str) -> Optional[int]:
    """'제72조' → 72 로 변환"""
    if s is None:
        return None
    m = re.search(r"\d+", str(s))
    return int(m.group()) if m else None

def extract_law_from_question(q: str) -> Tuple[Optional[str], Optional[int]]:
    if not q:
        return None, None
    q = q.strip()

    law_name = None
    m_name = LAW_NAME_RE.search(q)
    if m_name:
        law_name = m_name.group(1).strip()

    law_no = None
    m_art = ARTICLE_RE.search(q)
    if m_art:
        try:
            law_no = int(m_art.group(1))
        except Exception:
            law_no = None

    return law_name, law_no


# ===== 데이터 불러오기 =====
input_path = "/root/workspace/data/laws_by_article_portion.jsonl"

corpus = {}
with open(input_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f, 1):
        if not line.strip():
            continue
        obj = json.loads(line)

        raw_name = obj.get("law_name", "")
        raw_no   = obj.get("law_no", None)

        corpus[f"doc{idx}"] = {
            "title": f"{raw_name} {obj.get('title','')}",
            "text": obj.get("text", ""),
            "law_name": raw_name,                     # 원본
            "law_name_norm": normalize(raw_name),     # ✅ 정규화(조합/분해 차이 해결)
            "law_no_str": str(raw_no) if raw_no is not None else None,
            "law_no_int": extract_digits_to_int(raw_no),  # ✅ '제72조' -> 72
        }

print(f"불러온 문서 수: {len(corpus)}")


# ===== 테스트용 질문 =====
questions = [
    "개인정보보호법 제28조에 따르면 무엇을 해야 합니까?",
    "정보통신망법 제48조에 따르면 어떤 의무가 있나요?",
    "신용정보법 제32조는 무엇을 규정하나요?",
    "보험업법 제100조는 어떤 내용을 담고 있나요?",
    "전자서명법 제3조에 따른 요건은 무엇입니까?",
    "전자금융거래법 제6조는 무엇을 규정하나요?",
]

# ===== 테스트 실행 =====
for q in questions:
    law_name, law_no = extract_law_from_question(q)
    print(f"\nQ: {q}")
    print(f" → 추출 결과: (법령명: {law_name}, 조문번호: {law_no})")

    qname_norm = normalize(law_name) if law_name else None

    matched = [
        (doc_id, doc)
        for doc_id, doc in corpus.items()
        if qname_norm and (qname_norm in doc["law_name_norm"])    # ✅ doc의 law_name이 추출된 law_name(정규화)을 포함
        and (law_no is None or doc["law_no_int"] == law_no)        # ✅ 정수 동등 비교
    ]

    if matched:
        print(f" → corpus 매칭된 문서 수: {len(matched)}")
        for doc_id, doc in matched[:5]:
            print(f"   - {doc_id}: {doc['title']} (law_no={doc['law_no_str']})")
    else:
        print(" → corpus 매칭 없음")

불러온 문서 수: 1634

Q: 개인정보보호법 제28조에 따르면 무엇을 해야 합니까?
 → 추출 결과: (법령명: 개인정보보호법, 조문번호: 28)
 → corpus 매칭된 문서 수: 12
   - doc26: 개인정보 보호법 개인정보취급자에 대한 감독 (law_no=제28조)
   - doc27: 개인정보 보호법 상호주의 (law_no=제28조의10)
   - doc28: 개인정보 보호법 준용규정 (law_no=제28조의11)
   - doc29: 개인정보 보호법 가명정보의 처리 등 (law_no=제28조의2)
   - doc30: 개인정보 보호법 가명정보의 결합 제한 (law_no=제28조의3)

Q: 정보통신망법 제48조에 따르면 어떤 의무가 있나요?
 → 추출 결과: (법령명: 정보통신망법, 조문번호: 48)
 → corpus 매칭된 문서 수: 6
   - doc1459: 정보통신망 이용촉진 및 정보보호 등에 관한 법률(정보통신망법) 정보통신망 침해행위 등의 금지 (law_no=제48조)
   - doc1460: 정보통신망 이용촉진 및 정보보호 등에 관한 법률(정보통신망법) 침해사고의 대응 등 (law_no=제48조의2)
   - doc1461: 정보통신망 이용촉진 및 정보보호 등에 관한 법률(정보통신망법) 침해사고의 신고 등 (law_no=제48조의3)
   - doc1462: 정보통신망 이용촉진 및 정보보호 등에 관한 법률(정보통신망법) 침해사고의 원인 분석 등 (law_no=제48조의4)
   - doc1463: 정보통신망 ᄋ

In [ ]:
## 부모 자식 실험
from retrieval_parent import build_child_corpus, ExpansionIndex,HybridSearcher, DenseRetrieval, ParentAwareReranker


# 1) 코퍼스
corpus = build_child_corpus(args.child_jsonl)

# 2) 확장/부모 색인
xp = ExpansionIndex(args.expanded_jsonl or None, args.articles_jsonl or None)

# 3) 쿼리
queries = {}
if args.queries_path:
    # 형식: {"qid":"질문", ...} 또는 jsonl [{"id":"q1","text":"..."}, ...]
    import json, os
    if args.queries_path.endswith(".jsonl"):
        with open(args.queries_path,"r",encoding="utf-8") as f:
            for i,line in enumerate(f,1):
                try:
                    obj=json.loads(line)
                    qid = obj.get("id") or obj.get("qid") or f"q{i}"
                    queries[qid]=obj.get("text") or obj.get("query") or ""
                except: pass
    else:
        with open(args.queries_path,"r",encoding="utf-8") as f:
            obj=json.load(f)
            if isinstance(obj, dict):
                queries = obj
            elif isinstance(obj, list):
                for i,o in enumerate(obj,1):
                    qid = o.get("id") or o.get("qid") or f"q{i}"
                    queries[qid]=o.get("text") or o.get("query") or ""
elif args.query:
    queries = {"q1": args.query}
else:
    raise SystemExit("Query is required: --query or --queries_path")

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/seojeonghun/Desktop/금융 AI 챌린지/main/workspace_0827/data/test.csv")

